In [1]:
import pandas as pd
import re
import sys
import numpy as np

## Assuming all official roll numbers start with 0818
and 
# "identifier","github_username","github_id","name" 
these columns exist in Github classroom roster CSV
# "Name","Enrollment"
these columns exist in Classroom's official record CSV

In [2]:
class_name = "IOT SEM 3 python"
gitClass_file_path , classData_file_path = "IOT Sem 3 Github.csv" , 'IOT Sem 3 official.csv'

##Assumming github usernames are in form BR123456-FirstLast
where BR123456 reprents rollnumber "0818BR123456"
and name of student is "First Last"

In [3]:

def is_valid_username(username):
    # Regular expression pattern for the username format BR123456-FirstLast.
    pattern = r"^[A-Z]{2}\d{6}-[A-Z][a-z]+[A-Z][a-z]+$"
    # Use the re.match function to check if the username matches the pattern.
    if re.match(pattern, username):
        return True
    else:
        return False



In [18]:
def separate_parts(username):
    # Define a regular expression pattern to extract enrollment and name parts.
    pattern = r"^([A-Z]{2}\d{6})-([A-Z][a-z]+)([A-Z][a-z]+)$"
    
    # Use re.match to find the parts.
    match = re.match(pattern, username)
    
    if match:
        enrollment_part = match.group(1)
        first_name = match.group(2)
        last_name = match.group(3)
        # Split the name part into first and last names.
        
        
        return enrollment_part, first_name, last_name
    else:
        return None, None, None


## Assumming That there are no records with same roll number

should Find way around for scenerio where students may use same roll numbers in their username 

In [5]:
def no_mid_name(full_name):
    # Split the full name into words
    name_parts = full_name.split()
    
    # Extract the first name and last name
    first_name = name_parts[0]
    last_name = name_parts[-1]
    
    return first_name, last_name


In [6]:
def search_and_match(enrollment, name, surname, dataframe):
    status_flags = {
        'enrollment_not_in_records': False,
        'name_mismatch': False,
        'match_successful': False,
        'correct_name': None,
    }

    # Check if the enrollment number exists in the records
    if enrollment not in dataframe['Enrollment'].values:
        status_flags['enrollment_not_in_records'] = True
        return status_flags
    
    # Retrieve the record for the given enrollment number
    student_record = dataframe[dataframe['Enrollment'] == enrollment].iloc[0]
    record_name = student_record['Name']
    
    # Compare provided name and surname with the records
    name = name+' '+surname
    name = name.upper()
    fnm , lnm = no_mid_name( record_name )
    record_name = fnm+' '+lnm
    record_name = record_name.upper() 
    if name != record_name:
        status_flags['name_mismatch'] = True
        status_flags['correct_name'] = record_name
    if not status_flags['name_mismatch']:
        status_flags['match_successful'] = True
    
    return status_flags


In [7]:

def search_name(name, official_df):
    record_names = official_df['Name']
    filtered_data = record_names[record_names == name]
    if filtered_data.empty:
        return False
    return True


data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva']
}

df = pd.DataFrame(data)
    

#### Test the function
name_to_search = 'Raju'
result = search_name(name_to_search, df)

print(f"Is '{name_to_search}' in the DataFrame? {result}")

In [8]:

def find_miscellaneous_names(official_df, platform_df):
    miscellaneous_names = []
    for index, row in platform_df.iterrows():
        name = row['identifier']
        if not search_name(name, official_df):
            miscellaneous_names.append(name)
    
    # Create a new dataframe for miscellaneous names
    miscellaneous_df = pd.DataFrame({'Miscellaneous Names': miscellaneous_names})
    
    return miscellaneous_df



In [9]:
# Function to match identifier with name and surname
def match_identifier(username, surname, identifier):
    t = username+' '+surname
    t = t.upper()
    identifier = identifier.upper()
    return (t==identifier)


In [10]:
def add_row_dataframe(df, data_dict):
    """
    Add a new row to the given DataFrame based on the provided data_dict.
    
    Args:
        df (pd.DataFrame): Original DataFrame.
        data_dict (dict): Dictionary containing column names as keys and corresponding values.
        
    Returns:
        pd.DataFrame: Updated DataFrame with the new row added.
    """
    # Creating a new DataFrame from the data_dict
    new_row_df = pd.DataFrame([data_dict])
    
    # Concatenating the original DataFrame and the new row DataFrame
    updated_df = pd.concat([df, new_row_df], ignore_index=True)
    return updated_df


## This is the main runner

In [31]:
# Function to process CSV files
def validate_records(gitClass_file_path , classData_file_path ,class_name):
    try:
        # Read CSV files into a pandas DataFrame
        gitClass_df = pd.read_csv(gitClass_file_path)
        classData_df = pd.read_csv(classData_file_path)
        # Checking miscellaneous names in Google classroom
        
        miscellaneous_names = find_miscellaneous_names(classData_df, gitClass_df)
        print("miscellaneous_names in Google classroom which are not in official records")
        print(miscellaneous_names)
        file_name = class_name+' miscellaneous_names.csv'
        miscellaneous_names.to_csv(file_name, index=False)
        print("saved to "+file_name)
        
        # Checking discipencies in github class room data
        improp_df = {
            'Enrollment': [],
            'UserName': [],
            'Name': [],
            'NotJoinedPlatform': [],
            'ImproperUserName': [],
            'WrongIdentifier': [],
            'EnrollmentNotInRecords': [],
            'NameMismatch': [],
        }
        
        # Creating a DataFrame
        improp_df = pd.DataFrame(improp_df)
        
    
        
        for index, row in gitClass_df.iterrows():
            
            username = row['github_username']
            identifier = row['identifier']

            improp_dict = {
                'Enrollment': None,
                'UserName': None,
                'Name': None,
                'NotJoinedPlatform': False,
                'ImproperUserName': False,
                'WrongIdentifier': False,
                'EnrollmentNotInRecords': False,
                'NameMismatch': False,
            }
            # Checks if student has not joined the platform
            # NotJoinedPlatform means Not Joined Platform
            if type(username)==float and np.isnan(username) :
                print(f"Unlinked username for identifier: {identifier}")
    
                improp_dict['Name'] = identifier
                improp_dict['NotJoinedPlatform']=True
                improp_df = add_row_dataframe(improp_df , improp_dict)
                
                continue
            # Checks proper name convention of student
            # ImproperUserName means student has not followed name convention properly while creating github account
            if not is_valid_username(username):
                print(f"Improper name convention for username: {username}")
                
                
                improp_dict['UserName'] = username
                improp_dict['ImproperUserName'] = True
                improp_df = add_row_dataframe(improp_df , improp_dict)
                
                continue
            
            # Extracts name and surname from username
            enrollment,name, surname = separate_parts(username) # Extract name and surname from username
            enrollment="0818"+enrollment

            # Matchs linked identifier with extracted name and surname
            # WrongIdentifier means student has probably linked with wrong identifer
            if not match_identifier(name, surname, identifier):
                print(f"Identifier mismatch for username: {username}")
                
                improp_dict['UserName'] = username
                improp_dict['WrongIdentifier'] = True
                improp_df = add_row_dataframe(improp_df , improp_dict)
                
                continue
                
            # Checking with official records
            status_flags = search_and_match(enrollment,name, surname, classData_df )
            
            # checks if the extracted enrollment is in records or not
            # EnrollmentNotInRecords means username's enrollment is not found in official records
            if status_flags['enrollment_not_in_records']:
                print(f"enrollment not in records: {enrollment} of username: {username}")
                
                improp_dict['UserName'] = username
                improp_dict['Enrollment'] = enrollment
                improp_dict['EnrollmentNotInRecords'] = True
                improp_df = add_row_dataframe(improp_df , improp_dict)
                
                continue
            # checks if the extracted enrollment corresponds to name in the records
            # NameMismatch means there is a mismatch between username's name and the official record name corresponding to the provided enrollment
            if status_flags['name_mismatch']:
                print(f"enrollment : {enrollment} does not correspond to name : {(name+' '+surname)} instead it does to : {(status_flags['correct_name'])}")
                
                improp_dict['UserName'] = username
                improp_dict['Name'] = status_flags['correct_name']
                improp_dict['Enrollment'] = enrollment
                improp_dict['NameMismatch'] = True
                improp_df = add_row_dataframe(improp_df , improp_dict)
                
                continue
        print('following faulty records found')
        print(improp_df)
        
        file_name = class_name+' Improper Records.csv'
         # saving the excel
        improp_df.to_csv(file_name, index=False)
        print("saved to "+file_name)
        return improp_df
    except KeyError as e:
        print(f"{e} related error was encounterd during data access and handling. Provided files' columns are not in proper format")
        return None
    except FileNotFoundError as e:
        print(f"{e} . Provide correct name or path to the files")
        return None
    #except Exception as e:
        print(f"Error: {e}")
        return None


# MAIN CALL

In [32]:
    

faulty_records_df = validate_records(gitClass_file_path , classData_file_path, class_name)

miscellaneous_names in Google classroom which are not in official records
      Miscellaneous Names
0            AKSHAT JHALA
1             ANAND TOMAR
2            Aaniya Mehra
3              Aman Nayak
4                Aman Pal
5           HARSH VARDHAN
6             KP YouTuber
7                 KisnaOP
8           Krish Dabhade
9   Mo. Ashfaque Mansoori
10          SHAIKH KAMDAR
11         SHIVAM CHOUHAN
12     SHIVENDRA PATERIYA
13          Shubham Gwale
saved to IOT SEM 3 python miscellaneous_names.csv
Unlinked username for identifier: AADARSH TRIPATHI
Unlinked username for identifier: AKSHAT JHALA
Unlinked username for identifier: AMAN PATEL
Improper name convention for username: IS221010-AMOLPAWAR
Improper name convention for username: IS221011-ANAND
Improper name convention for username: IS221015-Ayushimakodiya
Unlinked username for identifier: Aaniya Mehra
Unlinked username for identifier: Aman Nayak
Unlinked username for identifier: Aman Pal
Improper name convention for user

# Share with students

In [33]:
def group_faulty_records(dataframe):
    faulty_records_dict = {}
    for index, row in dataframe.iterrows():
        enrollment = row['Enrollment']
        UserName = row['UserName']
        Name = row['Name']
        problems = []
        if bool(row['NotJoinedPlatform']):
            problems.append('Not Joined Platform')
        if bool(row['ImproperUserName']):
            problems.append('Improper Name Convention')
        if bool(row['WrongIdentifier']):
            problems.append('Linked with Wrong Identifier')
        if bool(row['EnrollmentNotInRecords']):
            problems.append('Enrollment Not Found in Records')
        if bool(row['NameMismatch']):
            problems.append('Name Mismatch with Official Record')
        
        for problem in problems:
            if problem not in faulty_records_dict:
                faulty_records_dict[problem] = []
            faulty_records_dict[problem].append(f"Name: {Name}, Enrollment: {enrollment}, UserName: {UserName}")
    
    return faulty_records_dict


# Calling the function to get grouped faulty records as a dictionary
grouped_faulty_records = group_faulty_records(faulty_records_df)

# Printing the grouped faulty records
print('Faulty records for '+class_name)
print("*" * len('Faulty records for '+class_name))
for problem, records in grouped_faulty_records.items():
    print(f"Problem: {problem}")
    print("_" * len(f"Problem: {problem}"))
    for record in records:
        print(record)
    print("-" * 30)

Faulty records for IOT SEM 3 python
***********************************
Problem: Not Joined Platform
____________________________
Name: AADARSH TRIPATHI, Enrollment: None, UserName: None
Name: AKSHAT JHALA, Enrollment: None, UserName: None
Name: AMAN PATEL, Enrollment: None, UserName: None
Name: Aaniya Mehra, Enrollment: None, UserName: None
Name: Aman Nayak, Enrollment: None, UserName: None
Name: Aman Pal, Enrollment: None, UserName: None
Name: GOVINDA LOVANSHI, Enrollment: None, UserName: None
Name: JAY VAISHNAV, Enrollment: None, UserName: None
Name: KARTIK DESHMUKH, Enrollment: None, UserName: None
Name: KP YouTuber, Enrollment: None, UserName: None
Name: KisnaOP, Enrollment: None, UserName: None
Name: Krish Dabhade, Enrollment: None, UserName: None
Name: MANAS LOKHANDE, Enrollment: None, UserName: None
Name: MEENAKSHI CHOUHAN, Enrollment: None, UserName: None
Name: ROHAN SINGH, Enrollment: None, UserName: None
Name: RUCHITA NIMBHORKAR, Enrollment: None, UserName: None
Name: SATYAM

In [14]:
df=pd.read_csv(gitClass_file_path)


In [15]:
df1=pd.read_csv(classData_file_path)

In [16]:
print("gitClass")
df

gitClass


,identifier,github_username,github_id,name
0,AADARSH TRIPATHI,NaN,NaN,NaN
1,ADITYA CHHAPARWAL,IS221003-AdityaChhaparwal,145006027.0,NaN
2,AKSHAT JHALA,NaN,NaN,NaN
3,AMAN PATEL,NaN,NaN,NaN
4,AMAN PATIDAR,IS221008-AmanPatidar,145003905.0,NaN
...,...,...,...,...
56,TANU PATIDAR,IS221067-TanuPatidar,145004160.0,NaN
57,VATSAL AGAL,NaN,NaN,NaN
58,VIJAY SINGH,IS221070-vijaysingh,145004109.0,NaN
59,VIVEK SHARMA,IS221071-VivekSharma,145004978.0,NaN


In [17]:
print("classData_file_path")
df1

classData_file_path


,S No,Name,Enrollment
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,1.0,AADARSH TRIPATHI,0818IS221001
3,2.0,AANIYA MEHRA,0818IS221002
4,3.0,ADITYA CHHAPARWAL,0818IS221003
...,...,...,...
69,68.0,TUSHAR,0818IS221068
70,69.0,VATSAL AGAL,0818IS221069
71,70.0,VIJAY SINGH,0818IS221070
72,71.0,VIVEK SHARMA,0818IS221071
